In [15]:
from typing import Any, List, Union
import logging
import re
from pprint import pprint
from jaf import path, utils
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [16]:
data2 = [
   {
      'name': 'Alice',
      'roles': ['admin', 'user'],
      'test': {
         'test2': {
            'test3': {
               'emails': ['alicedeep@example.com', ['bobdeep@test.com', 'joedeep@test.com']]
            }
         }
      },
      'profile': {
         'age': 30,
         'emails': ['alice@example.com', 'alice.work@example.com']
      }
   },
   {
      'name': 'Bob',
      'roles': ['user'],
      'test': {
         'test2': {
            'test3': {
               'emails': ['bob@mymail.com']
            }
         }
      }
   }
]

data = data2[1]

In [17]:
vals = path.path_values(path='**.emails', obj=data)
pprint(vals)

[PathValue(value=['bob@mymail.com'], path=['$', 'test', 'test2', 'test3', 'emails'])]


In [18]:
f = utils.wrap(func=lambda x, obj: x[0].upper())
results = f(vals, obj=data)
pprint(results)

DEBUG:jaf.utils:wrap called with func=<function <lambda> at 0x7aaff0d08c20>, func_name=λ x, obj ↦ x[0].upper())
DEBUG:jaf.utils:func_name='λ x, obj ↦ x[0].upper())' applied to args=([PathValue(value=['bob@mymail.com'], path=['$', 'test', 'test2', 'test3', 'emails'])],)
DEBUG:jaf.utils:new-list-path-value with arg [['bob@mymail.com']]
DEBUG:jaf.utils:apply λ x, obj ↦ x[0].upper()) to [['bob@mymail.com']] -> BOB@MYMAIL.COM
DEBUG:jaf.utils:results: ['BOB@MYMAIL.COM']


['BOB@MYMAIL.COM']


In [19]:
def length(x, obj):
   if isinstance(x, (str, list)):
      return len(x)
   return -1


logger.setLevel(logging.DEBUG)

f1 = utils.wrap(func=lambda x, obj: x.lower())
results1 = f1(results, obj=data)
f2 = utils.wrap(func=length)
results2 = f2(results1, obj=data)
pprint(results2)

DEBUG:jaf.utils:wrap called with func=<function <lambda> at 0x7aaff82fbba0>, func_name=λ x, obj ↦ x.lower())
DEBUG:jaf.utils:func_name='λ x, obj ↦ x.lower())' applied to args=(['BOB@MYMAIL.COM'],)
DEBUG:jaf.utils:new-list-path-value with arg ['BOB@MYMAIL.COM']
DEBUG:jaf.utils:apply λ x, obj ↦ x.lower()) to ['BOB@MYMAIL.COM'] -> bob@mymail.com
DEBUG:jaf.utils:results: ['bob@mymail.com']
DEBUG:jaf.utils:wrap called with func=<function length at 0x7aaff0d09080>, func_name=length
DEBUG:jaf.utils:func_name='length' applied to args=(['bob@mymail.com'],)
DEBUG:jaf.utils:new-list-path-value with arg ['bob@mymail.com']
DEBUG:jaf.utils:apply length to ['bob@mymail.com'] -> 14
DEBUG:jaf.utils:results: [14]


[14]


In [20]:
def two(x, obj):
   return [x * 2, x + 2]

f3 = utils.wrap(func=two)
results3 = f3(results2, obj=data)
pprint(results3)

DEBUG:jaf.utils:wrap called with func=<function two at 0x7aaff8123060>, func_name=two
DEBUG:jaf.utils:func_name='two' applied to args=([14],)
DEBUG:jaf.utils:new-list-path-value with arg [14]
DEBUG:jaf.utils:apply two to [14] -> [28, 16]
DEBUG:jaf.utils:results: [[28, 16]]


[[28, 16]]


In [21]:
# let's have two arguments

def add(x, y, obj):
   return x + y

f4 = utils.wrap(func=add)
results4 = f4(results3[0][0], results3[0][1], obj=data)

DEBUG:jaf.utils:wrap called with func=<function add at 0x7aaff8122d40>, func_name=add
DEBUG:jaf.utils:func_name='add' applied to args=(28,)
ERROR:jaf.utils:Error applying add to (28,): 'int' object is not iterable, skipping...
DEBUG:jaf.utils:results: []
